In [ ]:
import pandas as pd
import requests
from pprint import pprint
import time

In [ ]:
per_page = "50"
job_text = "Data Science"
params = {"text": job_text, "per_page": per_page}
res = requests.get('https://api.hh.ru/vacancies/', params=params )
print(res)

<Response [200]>


In [ ]:
res_json = res.json()
# res_content = res.content()
# res_text = res.text()
len(res_json)

8

In [ ]:
pprint(res_json)

In [ ]:
page = res_json['page']
pages = res_json['pages']
per_page = res_json['per_page']
found = res_json['found']

In [ ]:
page

0

In [ ]:
list_id = []
list_url = []
list_name = []

for i in range(page, pages+1):
  params = {"text": "Data Science", "per_page": "50", "page": f"{i}"}
  response = requests.get('https://api.hh.ru/vacancies/', params=params )
  response = response.json()
  time.sleep(0.5)
  for ii in response['items']:
    list_id.append(ii["id"])
    list_url.append(ii["url"])
    list_name.append(ii["name"])



In [ ]:
# print(list_id)
# print(len(list_url))
# print(list_name)


In [ ]:
response = requests.get(list_url[2]).json()
pprint(response)
# 'experience'
# 'key_skills'
# 'salary'
# 'alternate_url'
# 'area'
# 'employer'
#

In [ ]:
print(list_url[0])

https://api.hh.ru/vacancies/77874929?host=hh.ru


In [ ]:
# Вакансии Data Science только по Москве
# params = {"text": "Data Science", "per_page": "50", "area": "1"}
# res_mos = requests.get('https://api.hh.ru/vacancies/', params=params )
# if response == "<Response [200]>":
#   print(200)
# print(res_mos.status_code)

In [ ]:
import numpy as np
ind = 0
df_job = pd.DataFrame(columns = (['name',
                                  'experience',
                                  'key_skills',
                                  'salary',
                                  'alternate_url',
                                  'area',
                                  'employer',
                                  'professional_roles',
                                  'published_at',
                                  'description',
                                  'schedule',
                                  'employment']))

for item in list_url:
  response = requests.get(item)
  if response.status_code == 200:
    response = response.json()
    df_job.loc[ind] = {'name': response['name'],
                      'experience': response['experience']['name'],
                      'key_skills': [n['name'] for n in response['key_skills']],
                      'salary': response['salary'],
                      'alternate_url': response['alternate_url'],
                      'area': response['area']['name'],
                      'employer': response['employer']['name'],
                      'professional_roles': response['professional_roles'][0]['name'],
                      'published_at': response['published_at'],
                      'description': response['description'],
                      'schedule': response['schedule']['name'],
                      'employment': response['employment']['name']}
    ind += 1
    # time.sleep(0.2)
  else:
    print('Что-то пошло не так')


In [ ]:
# df_job.to_csv('jobs.csv', encoding = "utf-8-sig")
df_job_copy = df_job.copy(deep=True)
df_job_copy.info()

In [ ]:
def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'

df_job_style = df_job.style.format({'alternate_url': make_clickable})

In [ ]:
df_job_style

In [ ]:
df_job['experience'].value_counts()

От 3 до 6 лет         394
От 1 года до 3 лет    333
Более 6 лет            89
Нет опыта              69
Name: experience, dtype: int64

In [ ]:
df_job_to_me = df_job[(df_job['experience'] == 'Нет опыта') & (df_job['area'] == 'Москва')]
df_job_to_me